## Livability 

### Import Libraries

In [254]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from pyspark.sql import functions as F  #filtering
import pandas as pd
import matplotlib.pyplot as plt

### Start a Spark session

In [255]:
spark = (
    SparkSession.builder.appName('Livability Analysis')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "4g")
    .getOrCreate()
)

In [256]:
feature = pd.read_parquet('../data/curated/features_domain.parquet')

feature.head()

,url,price,address,property_type,latitude,longitude,Beds,Baths,Parking,bond,extracted_price,geometry,index_right,sa2_code,sa2_name,chg_flag,chg_lbl,sa3_code,sa3_name,sa4_code,sa4_name,gcc_code,gcc_name,ste_code,ste_name,aus_code,aus_name,areasqkm,loci_uri,geometry_proj,suburb,property_id,nearest_station_id,train_station_distance_km,cbd_distance_km,nearest_hospital_id,nearest_hospital_distance,mean_stores,total_stores,count_shopping_centres,SA2 code_x,Region,ERP_2021,ERP_2026,ERP_2031,ERP_2036,PNPD_2021,PNPD_2026,PNPD_2031,PNPD_2036,POPD_2021,POPD_2026,POPD_2031,POPD_2036,OPD_2021,OPD_2026,OPD_2031,OPD_2036,HHS_2021,HHS_2026,HHS_2031,HHS_2036,SPD_2021,SPD_2026,SPD_2031,SPD_2036,OCC_2021,OCC_2026,OCC_2031,OCC_2036,population_growth_2021_2026,population_growth_2026_2031,population_growth_2031_2036,total_population_growth,Facility Count,erp_2022_no.,erp_2023_no.,erp_change_no.,erp_change_percentage,pop_change_natural_increase,pop_change_net_internal_migration,pop_change_oversea_migration,area_km2,population_density_persons_per_km2_2023,property_sa2_name,nearest_parkres_distance,nearest_parkres_name,parkres_sa2_name,closest_school,distance_to_closest_school,closest_independent_school,distance_to_closest_independent_school,total_value_of_items,total_number_of_items,crime_frequency,SA2 code_y,SA2 name,Earners,Median age of earners,Sum,Median,Mean,P80/P20,P80/P50,P20/P50,P10/P50,Gini coefficient,Top 1%,Top 5%,Top 10%,Lowest Quartile,Second Quartile,Third Quartile,Highest Quartile
0,https://www.domain.com.au/10-allara-court-donv...,"$1,400.00","10 Allara Court, Donvale VIC 3111",Townhouse,-37.77,145.18,4.00,3.00,2.00,9125.00,1400.00,POINT (145.1811258 -37.77427300000001),310.00,211021261,Donvale - Park Orchards,0,No change,21102,Manningham - East,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.80,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (989259.4173701586 -4291898.086618988),Donvale,0,19900,5.66,25.93,H5063,1.92,NaN,NaN,NaN,211021261,Donvale - Park Orchards,16410.00,16758.34,17104.79,17452.53,268.03,295.81,329.73,361.21,16141.97,16462.53,16775.06,17091.31,5719.33,5867.52,6059.52,6251.52,2.82,2.81,2.77,2.73,5913,6112.00,6312.00,6512.00,0.97,0.96,0.96,0.96,2.12,2.07,2.03,6.22,NaN,16405,16511,106,0.60,-77,-59,242,20.80,793.70,Donvale - Park Orchards,3541.03,Mullum Mullum Park,Donvale - Park Orchards,1759,0.35,1724,1.09,NaN,NaN,NaN,211021261,Donvale - Park Orchards,10426,48,872950324,53881,83728,6.95,2.24,0.32,0.11,0.56,13.10,27.90,39.70,28.50,22.10,19.00,30.40
1,https://www.domain.com.au/7-pine-ridge-donvale...,$750 per week,"7 Pine Ridge, Donvale VIC 3111",House,-37.79,145.18,4.00,2.00,0.00,3259.00,750.00,POINT (145.1756489 -37.7912513),310.00,211021261,Donvale - Park Orchards,0,No change,21102,Manningham - East,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.80,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (988607.8235432974 -4293742.529802319),Donvale,1,19899,4.69,23.78,H5063,0.38,NaN,NaN,NaN,211021261,Donvale - Park Orchards,16410.00,16758.34,17104.79,17452.53,268.03,295.81,329.73,361.21,16141.97,16462.53,16775.06,17091.31,5719.33,5867.52,6059.52,6251.52,2.82,2.81,2.77,2.73,5913,6112.00,6312.00,6512.00,0.97,0.96,0.96,0.96,2.12,2.07,2.03,6.22,NaN,16405,16511,106,0.60,-77,-59,242,20.80,793.70,Donvale - Park Orchards,2536.54,Mullum Mullum Park,Donvale - Park Orchards,5019,0.99,1724,2.47,NaN,NaN,NaN,211021261,Donvale - Park Orchards,10426,48,872950324,53881,83728,6.95,2.24,0.32,0.11,0.56,13.10,27.90,39.70,28.50,22.10,19.00,30.40
2,https://www.domain.com.au/20-mulsanne-way-donv...,$1300 per week,"20 Mulsanne Way, Donvale VIC 3111",House,-37.80,145.18,5.00,2.00,2.00,5649.00,1300.00,POINT (145.1812636 -37.7972323),310.00,211021261,Donvale - Park Orchards,0,No change,21102,Manningham - East,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.80,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (989043.8745826276 -4294451.666

In [257]:
'''# Display all column names as a list (with no truncation)
columns_list = feature.columns.tolist()
for col in columns_list:
    print(col)'''

'# Display all column names as a list (with no truncation)\ncolumns_list = feature.columns.tolist()\nfor col in columns_list:\n    print(col)'

In [258]:
print(feature.columns)

Index(['url', 'price', 'address', 'property_type', 'latitude', 'longitude',
       'Beds', 'Baths', 'Parking', 'bond',
       ...
       'P20/P50', 'P10/P50', 'Gini coefficient', 'Top 1%', 'Top 5%', 'Top 10%',
       'Lowest Quartile', 'Second Quartile', 'Third Quartile',
       'Highest Quartile'],
      dtype='object', length=114)


### Inspect datasets

In [259]:
parkres_data = pd.read_csv("../data/curated/parkres/parkres.csv", index_col=0)
parkres_data.head(1)

,name,sa2_name,postcode,geometry
1,Lilydale-Warburton Rail Trail,Yarra Valley,3139,POLYGON ((1034153.6182959272 -4293796.63110344...


In [260]:
property_lost_data = pd.read_csv('../data/curated/property_data/property_lost.csv', index_col=0)
property_lost_data.head(1)

,year,year ending,police region,local government area,postcode,suburb,location division,property item,number of items,value of items ($),sa2_name
1,2024,March,1 North West Metro,Banyule,3079,Ivanhoe,1 Residential,Car Accessories,14,2040.00,Ivanhoe


In [261]:
train_hospital_data = pd.read_parquet('../data/curated/final_train_hospital_cbd_dist_data')

train_hospital_data.head(1)

,url,price,address,property_type,latitude,longitude,Beds,Baths,Parking,bond,extracted_price,geometry,index_right,sa2_code,sa2_name,chg_flag,chg_lbl,sa3_code,sa3_name,sa4_code,sa4_name,gcc_code,gcc_name,ste_code,ste_name,aus_code,aus_name,areasqkm,loci_uri,geometry_proj,suburb,property_id,nearest_station_id,train_station_distance_km,cbd_distance_km,nearest_hospital_id,nearest_hospital_distance,log_nearest_station_distance,log_nearest_hospital_distance,log_cbd_distance
0,https://www.domain.com.au/10-allara-court-donv...,"$1,400.00","10 Allara Court, Donvale VIC 3111",Townhouse,-37.77,145.18,4.00,3.00,2.00,9125.00,1400.00,POINT (145.1811258 -37.77427300000001),310.00,211021261,Donvale - Park Orchards,0,No change,21102,Manningham - East,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.80,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (989259.4173701588 -4291898.086618988),Donvale,0,19900,5.66,25.93,H5063,1.92,1.90,1.07,3.29


In [262]:
recreation_data = pd.read_csv('../data/curated/recreation_cleaned.csv')

recreation_data.head(1)

,Facility Count,sa2_code,sa2_name
0,66,206051128.00,Albert Park


In [263]:
shopping_data = pd.read_csv('../data/curated/agg_shopping_centre.csv')

shopping_data.head(1)

,sa2_name,mean_stores,total_stores,count_shopping_centres
0,Airport West,158.00,158.00,1


### Liveable Index Metrics

#### 1. Parks and Reservations Count

In [264]:
# Group by 'sa2_name' to count the number of parks and reservations
parkres_count = parkres_data.groupby('sa2_name').size().reset_index(name='parkres_count')

parkres_count.head()

,sa2_name,parkres_count
0,Abbotsford,11
1,Alphington - Fairfield,1
2,Altona,2
3,Altona Meadows,2
4,Altona North,1


#### 2. Property Lost Frequency

In [265]:
# Disable scientific notation in pandas
pd.set_option('display.float_format', '{:.2f}'.format)

In [266]:
# Grouping by 'sa2_name' and aggregating the required metrics
property_lost_stats = property_lost_data.groupby('sa2_name').agg(
    crime_frequency=('sa2_name', 'size'),  
    total_value_lost=('value of items ($)', 'sum'),  
    total_items_lost=('number of items', 'sum') 
).reset_index()

# Sort by crime_frequency in descending order
property_lost_stats = property_lost_stats.sort_values(by='crime_frequency', ascending=False)

property_lost_stats.head()

,sa2_name,crime_frequency,total_value_lost,total_items_lost
187,West Footscray - Tottenham,3287,29285221.68,40017
126,North Melbourne,2848,117641963.71,201522
7,Ashwood - Chadstone,2703,28860046.34,44787
148,Roxburgh Park - North,2604,56605655.19,73377
176,Truganina - South West,2559,73155166.38,95074


#### 3. Hospital Count

In [267]:
# Count unique hospital IDs in each SA2
hospital_count = train_hospital_data.groupby('sa2_name')['nearest_hospital_id'].nunique().reset_index(name='hospital_count')

# Sort the result by 'hospital_count' in descending order
hospital_count = hospital_count.sort_values(by='hospital_count', ascending=False)

hospital_count.head()

,sa2_name,hospital_count
22,Blackburn,6
183,Oakleigh - Huntingdale,6
202,Richmond (South) - Cremorne,5
236,Surrey Hills (West) - Canterbury,5
25,Box Hill,5


#### 4. Transportation Accessibility

In [268]:
# Count unique transport station IDs in each SA2
transport_count = train_hospital_data.groupby('sa2_name')['nearest_station_id'].nunique().reset_index(name='station_count')

# Sort the result by 'station_count' in descending order
transport_count = transport_count.sort_values(by='station_count', ascending=False)

transport_count.head()

,sa2_name,station_count
44,Camberwell,8
34,Brunswick West,8
152,Malvern East,8
51,Caulfield - North,7
15,Balwyn North,7


#### 5. CBD Accessibility

In [269]:
# Calculate the average distance to the CBD for each SA2
cbd_accessibility = train_hospital_data.groupby('sa2_name')['cbd_distance_km'].mean().reset_index(name='avg_cbd_distance_km')

# Sort the result by 'avg_cbd_distance_km' in ascending order
cbd_accessibility = cbd_accessibility.sort_values(by='avg_cbd_distance_km', ascending=True)

cbd_accessibility.head()

,sa2_name,avg_cbd_distance_km
157,Melbourne CBD - North,0.86
158,Melbourne CBD - West,1.12
156,Melbourne CBD - East,1.16
46,Carlton,1.86
268,West Melbourne - Residential,1.88


#### 6. Recreation / Entertainment Facilities Count

In [270]:
recreation_count = recreation_data.copy()

# Move 'sa2_name' to the first position
cols = ['sa2_name'] + [col for col in recreation_count.columns if col != 'sa2_name']
recreation_count = recreation_count[cols]

# Rename the column 'Facility Count' to 'recreation_facility_count'
recreation_count.rename(columns={'Facility Count': 'recreation_facilities_count'}, inplace=True)

recreation_count.drop(columns=['sa2_code'], inplace=True)

# Sort the result by 'recreation_facility_count' in descending order
recreation_count = recreation_count.sort_values(by='recreation_facilities_count', ascending=False)

recreation_count.head()

,sa2_name,recreation_facilities_count
91,Seaford (Vic.),480
119,Yarra Valley,467
79,Point Cook - North West,462
83,Reservoir - South West,439
18,Caroline Springs,385


#### 7. Shopping Centres Count

In [271]:
# Make a copy of the shopping data
shopping_count = shopping_data.copy()

# Drop unnecessary columns 'mean_stores' and 'total_stores'
shopping_count.drop(columns=['mean_stores', 'total_stores'], inplace=True)

# Rename the column 'count_shopping_centres' to 'shopping_centres_count'
shopping_count.rename(columns={'count_shopping_centres': 'shopping_centres_count'}, inplace=True)

# Sort the result by 'shopping_centres_count' in descending order
shopping_count = shopping_count.sort_values(by='shopping_centres_count', ascending=False)

shopping_count.head()

,sa2_name,shopping_centres_count
84,Melbourne CBD - East,10
61,Glen Waverley - West,3
85,Melbourne CBD - North,3
137,Warrnambool - North,3
40,Docklands,3


#### 8. School Count

In [272]:
# Count unique closest schools in each SA2
school_count= feature.groupby('sa2_name')['closest_school'].nunique().reset_index(name='school_count')

# Sort the result by 'avg_cbd_distance_km' in descending order
school_count = school_count.sort_values(by='school_count', ascending=False)

school_count.head()

,sa2_name,school_count
265,Werribee - South,13
195,Preston - East,12
230,St Kilda East,11
14,Balwyn,11
178,North Melbourne,10


#### 9. Rent Count

In [273]:
# Calculate the average rent (price) by SA2
rent_count = feature.groupby('sa2_name').agg({'extracted_price': 'mean'}).reset_index()

# Sort the result by the average rent (price) in ascending order
rent_count = rent_count.sort_values(by='extracted_price', ascending=False)

rent_count.head()

,sa2_name,extracted_price
261,Warrandyte - Wonga Park,1700.00
18,Beaumaris,1104.69
29,Brighton East,1060.00
236,Surrey Hills (West) - Canterbury,1034.52
12,Aspendale Gardens - Waterways,1000.00


### Calculate Liveablity Index

In [274]:
# Merge all relevant data on 'sa2_name'
liveable_count = school_count.merge(transport_count, on='sa2_name', how='outer')\
    .merge(hospital_count, on='sa2_name', how='outer')\
    .merge(parkres_count, on='sa2_name', how='outer')\
    .merge(recreation_count, on='sa2_name', how='outer')\
    .merge(shopping_count, on='sa2_name', how='outer')\
    .merge(rent_count, on='sa2_name', how='outer')\
    .merge(property_lost_stats, on='sa2_name', how='outer')\
    .merge(cbd_accessibility, on='sa2_name', how='outer')

# Fill any NaN values with 0 for counting purposes
liveable_count.fillna(0, inplace=True)

In [275]:
liveable_count.head(1)

,sa2_name,school_count,station_count,hospital_count,parkres_count,recreation_facilities_count,shopping_centres_count,extracted_price,crime_frequency,total_value_lost,total_items_lost,avg_cbd_distance_km
0,Abbotsford,5.00,4.00,4.00,11.00,0.00,0.00,699.90,677.00,10978545.92,18992.00,4.64


In [276]:
# Define the weights for each factor (you can adjust these weights)
school_weight = 0.1
station_weight = 0.2
hospital_weight = 0.1
park_weight = 0.1
recreation_weight = 0.1
shopping_weight = 0.1
rent_weight = 0.1
crime_weight = 0.1
cbd_weight = 0.1

In [277]:
# Z-score normalization for each element
liveable_count['school_count'] = (liveable_count['school_count'] - liveable_count['school_count'].min()) / (liveable_count['school_count'].max() - liveable_count['school_count'].min())
liveable_count['station_count'] = (liveable_count['station_count'] - liveable_count['station_count'].min()) / (liveable_count['station_count'].max() - liveable_count['station_count'].min())
liveable_count['hospital_count'] = (liveable_count['hospital_count'] - liveable_count['hospital_count'].min()) / (liveable_count['hospital_count'].max() - liveable_count['hospital_count'].min())
liveable_count['parkres_count'] = (liveable_count['parkres_count'] - liveable_count['parkres_count'].min()) / (liveable_count['parkres_count'].max() - liveable_count['parkres_count'].min())
liveable_count['recreation_facilities_count'] = (liveable_count['recreation_facilities_count'] - liveable_count['recreation_facilities_count'].min()) / (liveable_count['recreation_facilities_count'].max() - liveable_count['recreation_facilities_count'].min())
liveable_count['shopping_centres_count'] = (liveable_count['shopping_centres_count'] - liveable_count['shopping_centres_count'].min()) / (liveable_count['shopping_centres_count'].max() - liveable_count['shopping_centres_count'].min())
liveable_count['extracted_price'] = (liveable_count['extracted_price'] - liveable_count['extracted_price'].min()) / (liveable_count['extracted_price'].max() - liveable_count['extracted_price'].min())
liveable_count['avg_cbd_distance_km'] = (liveable_count['avg_cbd_distance_km'] - liveable_count['avg_cbd_distance_km'].min()) / (liveable_count['avg_cbd_distance_km'].max() - liveable_count['avg_cbd_distance_km'].min())
liveable_count['crime_frequency'] = (liveable_count['crime_frequency'] - liveable_count['crime_frequency'].min()) / (liveable_count['crime_frequency'].max() - liveable_count['crime_frequency'].min())

In [278]:
# Calculate the liveability index based on the weighted sum of the normalized factors
liveable_count['liveability_index'] = (
    liveable_count['school_count'] * school_weight +
    liveable_count['station_count'] * station_weight +
    liveable_count['hospital_count'] * hospital_weight +
    liveable_count['parkres_count'] * park_weight +
    liveable_count['recreation_facilities_count'] * recreation_weight +
    liveable_count['avg_cbd_distance_km'] * cbd_weight +
    liveable_count['crime_frequency'] * crime_weight +
    liveable_count['shopping_centres_count'] * shopping_weight -
    liveable_count['extracted_price'] * rent_weight  # Rent is considered negative in liveability
)

In [284]:
# Sort by liveability index in descending order (best liveability first)
liveable_count = liveable_count.sort_values(by='liveability_index', ascending=False)

liveable_count[['sa2_name', 'liveability_index']].head(10)

,sa2_name,liveability_index
246,Richmond - North,0.37
121,Frankston,0.37
328,Yarra Valley,0.36
222,Oakleigh - Huntingdale,0.36
51,Camberwell,0.35
59,Caulfield - North,0.35
217,North Melbourne,0.35
178,Malvern East,0.34
261,Seaford (Vic.),0.34
177,Malvern - Glen Iris,0.31
